<a href="https://colab.research.google.com/github/Tetsujinfr/darknet/blob/master/VIBE_inference_TetsFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# build openpose STAF branch

import os
from os.path import exists, join, basename, splitext
#Oroginal openpose repo:
#git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
#STAF repo
git_repo_url = '-b staf https://github.com/soulslicer/openpose.git'

project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

In [0]:
#Install STAF models
%cd /content/openpose/models
!sh getModels.sh

In [0]:
# Clone the VIBE repo and install VIBE
%cd /content
!git clone https://github.com/mkocabas/VIBE.git
%cd VIBE/
# Install the other requirements
!pip install torch==1.4.0 numpy==1.17.5
!pip install enum34==1.1
!pip install -r requirements.txt

In [0]:
# Download pretrained weights and SMPL data to run VIBE
!sh prepare_data.sh

In [0]:
#Install BBOX multitracker
%cd /content/VIBE/
!pip install git+https://github.com/mkocabas/multi-person-tracker.git

In [0]:
# download the youtube you want to infer with the given ID
YOUTUBE_ID = 'EB_b8JVPuw8'
YouTubeVideo(YOUTUBE_ID)
!rm -rf youtube.mp4
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:23 -t 46 /content/video.mp4

In [0]:

# Run the VIBE demo
!python demo.py --vid_file /content/video.mp4 --output_folder output/ --tracking_method pose --run_smplify --staf_dir /content/openpose/ # temporal tracking
#!python demo.py --vid_file video.mp4 --output_folder output/ --tracking_method bbox --detector maskrcnn --wireframe --sideview # bounding box multitracker
# You may use --sideview flag to enable from a different viewpoint, note that this doubles rendering time.
# !python demo.py --vid_file sample_video.mp4 --output_folder output/ --sideview

In [0]:
# Play the generated video
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('output/video/video_vibe_result.mp4')

In [0]:
# Use the colab file explorer to download your video inference output